# Note Book for Capstone Project - IBM Data Science Professional Certification

In [12]:
import pandas as pd
import numpy as np

# Libraries for web scraping
from bs4 import BeautifulSoup
import requests
import csv

**Reading the details from wikipedia to fetch postal codes for Canada using BeautifulSoup package**

In [13]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

**Reading the web content into a csv file for reading directly into pandas dataframe**

In [14]:
data_table = soup.find('table', class_='wikitable sortable')
data_row = data_table.find_all('tr')

csv_file = open('canada_post_codes','w')
csv_writer = csv.writer(csv_file)

for line in data_row:
    p_ = line.text
    l_ = p_.split('\n')
    csv_writer.writerow([l_[1],l_[2],l_[3]])

csv_file.close()                         

In [15]:
df = pd.read_csv('canada_post_codes')

**Ignoring and removing all the Boroughs those are 'Not assigned'**

In [16]:
# Remove the rows with borough 'Not Assigned'
df = df[df.Borough!='Not assigned'].reset_index(drop=True)

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Replace all 'Not assigned' neighbourhoods with valid Boroughs in the dataframe**

In [18]:
## Replace all 'Not assigned' Neighborhoods with Borough

def neighborhood(row_):
    if row_['Neighbourhood'] == 'Not assigned': 
        x_ = row_['Borough']
    else:
        x_ = row_['Neighbourhood']
    return x_
    
for row in df.iterrows():
    df['Neighbourhood'] = df.apply(lambda row: neighborhood(row), axis=1)


**Build comma seperated neighbourhoods with common Boroughs under same postal code**

In [19]:
# if the value count>1 you need to merge the neighbourhood
def merge_nh(row):
    cnt_ = df[df.Postcode == row[1][0]].shape[0]  
    if cnt_ > 1 :
       x_ = df[df.Postcode == row[1][0]]['Neighbourhood'].tolist()
       l_ = ','.join(x_)
    else:
       l_ = row[1][2]
    return l_

k=[]
for row in df.iterrows():
    k.append(merge_nh(row))

df['Neighbourhood'] = k

**Removing all duplicates resulted from previous step**

In [20]:
df.drop_duplicates(keep='first',inplace=True)
df = df.reset_index(drop=True)

In [21]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


# **Shape of the dataframe**

In [22]:
# shape of the data frame
df.shape

(103, 3)

# End - Part of assignment Step 2 intended for week 3 (10 marks section)